In [ ]:
%load_ext google.cloud.bigquery

In [20]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json(r'C:\Users\amand\project\RA_WORK_CONTENT_BQ.json')
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'C:\Users\amand\project\RA_WORK_CONTENT_BQ.json'

import pandas as pd
table = '[ra-work-content.ceo_tweet_streaming.search_2017_14]'
import pandas_gbq
import logging

In [2]:
import tweepy
from tweepy import OAuthHandler
import sys
sys.path.append(r'C:\Users\amand\project')
import twitterkey
auth = OAuthHandler( twitterkey.consumer_key, twitterkey.consumer_secret)
auth.set_access_token(twitterkey.access_token, twitterkey.access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

## extract id of whom ceo reply to

In [21]:
sql_query="""
select string(reply_to_user_id) as reply_to_user_id
from(
select reply_to_user_id
from %s
where  reply_to_user_id  is not null
group by reply_to_user_id)
"""%(table)



query_config = bigquery.QueryJobConfig(use_legacy_sql=True)
ceo_reply = client.query(sql_query, job_config=query_config, location='US').to_dataframe()
ceo_reply.head()

,reply_to_user_id
0,1176592952
1,14159148
2,44438256
3,726470882
4,81124540


## getting description, screen name, verified of whom ceo reply to

### note: the def of screen name and name is the opposite of tweepy definition

In [5]:
tweetCount = 1
reply_cnt = 0
result_cnt = 0
no_twitter_cnt = 0
fail = 0
reply_to_id = []
reply_to_screen_name = []
reply_to_description = []
reply_to_verified = []
cnt = 0
for index, row in ceo_reply.iterrows():
    cnt += 1

    id = row['reply_to_user_id']
    reply_cnt += 1

    try:
        tweet = api.user_timeline(user_id=id, count=tweetCount, wait_on_rate_limit_notify=True, wait_on_rate_limit=True)
        if tweet == []:
            no_twitter_cnt += 1
            continue
        if result_cnt == 20:
            break
        else: 
            result_cnt += 1
            for line in tweet:
                reply_to_id.append(id)
                reply_to_screen_name.append(line.user.name)
                reply_to_description.append(line.user.description)
                reply_to_verified.append(line.user.verified)
                
    except tweepy.TweepError:
        fail += 1
        

In [6]:
    
rid = pd.Series(reply_to_id)   
scn = pd.Series(reply_to_screen_name)
des = pd.Series(reply_to_description)
ver =  pd.Series(reply_to_verified)
reply = pd.concat([rid, scn, des, ver, ], axis=1)
reply.columns =['reply_to_user_id', 'reply_to_user_screen_name', 'reply_to_user_description', 'reply_to_user_verified']
pd.set_option('display.max_colwidth', -1)
reply.head(10)


,reply_to_user_id,reply_to_user_screen_name,reply_to_user_description,reply_to_user_verified
0,1176592952,Bloom Burton & Co,Accelerating returns in healthcare,False
1,14159148,United Nations,Official account of #UnitedNations. Get latest information on the UN. Follow us on Instagram: unitednations. #GlobalGoals #StandUp4HumanRights,True
2,44438256,Matthew Herper,"Senior Writer, Medicine at STAT News. This is biology's century; Every data point has a face.",True
3,726470882,The Galien Foundation,A global Foundation that fosters the open exchange of ideas to drive scientific #innovation in medicine. #PrixGalien,False
4,81124540,Robert Langreth,"Science and health reporter at large, Bloomberg News",False
5,50393960,Bill Gates,Sharing things I'm learning through my foundation work and other interests.,True
6,14293310,TIME,Breaking news and current events from around the globe. Hosted by TIME staff.,True
7,7589852,JA Martinez Aguilar,,False
8,7429102,Mark Warner,"Serving Virginia in the US Senate. Vice Chairman, Intel Committee. Dad, Husband, Budget Wonk. Previous: Serial entrepreneur, business guy, 69th Governor of Va",True
9,181572333,Chance The Rapper Album in July,back in town finishing my debut owbum.,True


In [7]:

logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
reply.to_gbq('ceo_tweet_streaming.reply_to', 'ra-work-content')

TableCreationError: Could not create the table because it already exists. Change the if_exists parameter to 'append' or 'replace' data.

## getting description of ceo's follower 
## following change api and destination table

In [18]:
table = '[ra-work-content.ceo_tweet_streaming.account_2017_14]'

sql_query="""
select LTRIM(id_str, "'" ) as id 
from(
select  id_str 
from %s
group by id_str) 
"""%(table)

query_config = bigquery.QueryJobConfig(use_legacy_sql=True)
ceo = client.query(sql_query, job_config=query_config).to_dataframe()

ceo1 = ceo[0:100]
ceo2 = ceo[100:200]
ceo3 = ceo[200: ]

In [ ]:
c_id = []
f_id = []
f_description = []
f_name = []
f_screen_name = []
f_verified = []
f_following = []
fail = 0
cnt = 0

for index, row in ceo.iterrows():
    id = row['id']   
    try: 
        cnt += 1
        tweet = tweepy.Cursor(api.followers, user_id=id,  wait_on_rate_limit =True, wait_on_rate_limit_notify=True).items()
        #api.friends
        if tweet == []:
            continue
            
        else: 
            for line in tweet:
                c_id.append(id)
                f_id.append(line.id)
                f_name.append(line.screen_name)
                f_screen_name.append(line.name) 
                f_description.append(line.description) 
                f_verified.append(line.verified) 
                f_following.append(line.following) 

    except tweepy.TweepError:
        fail += 1

    
        

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1132669973824823296, 'id_str': '1132669973824823296', 'name': 'Adi', 'screen_name': 'Adi78275250', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 2, 'listed_count': 0, 'created_at': 'Sun May 26 15:29:00 +0000 2019', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 0, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_link_color'

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1129111734525022209, 'id_str': '1129111734525022209', 'name': 'Hashim Khan', 'screen_name': 'HashimK95874310', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 3, 'friends_count': 164, 'listed_count': 0, 'created_at': 'Thu May 16 19:49:50 +0000 2019', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 0, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1129112096426332160/aZVH9kDK_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1129112096426332160/aZVH9kDK_normal.jpg', 'profile_l

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1088100576079085568, 'id_str': '1088100576079085568', 'name': 'GeoDATA Forum', 'screen_name': 'GeoDataForum', 'location': 'London, England', 'description': 'The leading event for data specialists focused on location intelligence. Co-located with @GEOBusinessShow 21-22 May #gis #geospatial', 'url': 'https://t.co/AgsomBD4sG', 'entities': {'url': {'urls': [{'url': 'https://t.co/AgsomBD4sG', 'expanded_url': 'http://www.geodataforum.com', 'display_url': 'geodataforum.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 217, 'friends_count': 1281, 'listed_count': 3, 'created_at': 'Wed Jan 23 15:46:08 +0000 2019', 'favourites_count': 5, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 57, 'lang': None, 'status': {'created_at': 'Fri May 24 11:18:54 +0000 2019', 'id': 1131882259487838209, 'id_str': '1131882259487838209', 'text': 'The

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 2242054345, 'id_str': '2242054345', 'name': 'nazmus sakib', 'screen_name': 'sakib_fx', 'location': 'Dhaka,Bangladesh', 'description': '█║▌│║▌║▌│█│▌║│█║█║\n¹³²³ ³³¹² ²³³¹ ² ¹ ³²²³', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 293, 'friends_count': 161, 'listed_count': 3, 'created_at': 'Thu Dec 12 08:49:28 +0000 2013', 'favourites_count': 223, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 134, 'lang': None, 'status': {'created_at': 'Wed May 15 17:02:37 +0000 2019', 'id': 1128707267484737536, 'id_str': '1128707267484737536', 'text': "RT @DudeWhoCode: My DMs are flooded and as Twitter messaging sucks I have noted every handle in DMs. I'll get back to you to schedule a sho…", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'DudeWhoCode', 'name': 'Naren', 'id': 1342542518, 

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1107704097384398848, 'id_str': '1107704097384398848', 'name': 'Plastic 0%', 'screen_name': 'Plastic0PerCent', 'location': 'Levallois-Perret, France', 'description': 'Now We Can !\nPlastic 0% aims at promoting, supporting & sourcing existing or future recycling solutions for Plastics or PET (Polyethylene Terephthalate)', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 377, 'friends_count': 4568, 'listed_count': 4, 'created_at': 'Mon Mar 18 18:03:31 +0000 2019', 'favourites_count': 55, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 98, 'lang': None, 'status': {'created_at': 'Thu May 16 05:00:11 +0000 2019', 'id': 1128887849112297482, 'id_str': '1128887849112297482', 'text': 'Parce que les Statistiques et Projections montrent que la situation doit changer dés maintenant et plus que jamais… https://t.co/Q3ZuMF79FG', 'truncat

User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1114851265249521666, 'id_str': '1114851265249521666', 'name': 'Brianna', 'screen_name': 'Brianna12203364', 'location': '', 'description': 'I am an expert full-time graphic designer with 7 years of working \nexperience in Logo Design and Brand Identity in the graphic designing field.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 94, 'friends_count': 1407, 'listed_count': 1, 'created_at': 'Sun Apr 07 11:23:49 +0000 2019', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 2238, 'lang': None, 'status': {'created_at': 'Fri Apr 19 08:10:33 +0000 2019', 'id': 1119151285578555392, 'id_str': '1119151285578555392', 'text': '@jamie_genevieve Modern Minimal Logo for your Business or Website.\nCheck this out -  https://t.co/sVBAWwGm9a… https://t.co/EjnCopn2F7', 'truncated': True, 'entities': {'hashtags': [], '

Rate limit reached. Sleeping for: 354


User(_api=<tweepy.api.API object at 0x0000011A184D9EF0>, _json={'id': 1617755652, 'id_str': '1617755652', 'name': 'Steve Foerster', 'screen_name': 'stevehfoerster', 'location': 'Dominica, West Indies', 'description': 'President of @NewWorldU. Interests: #OER, #MOOC, #CBE, #GlobalEd, #mlearning, #EdTech, #Ed4Dev, #Dominica, #startups, #econdev, #blockchain, #vegan, and #DnD.', 'url': 'https://t.co/3QCHbmIns5', 'entities': {'url': {'urls': [{'url': 'https://t.co/3QCHbmIns5', 'expanded_url': 'http://stevefoerster.com', 'display_url': 'stevefoerster.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 482, 'friends_count': 1099, 'listed_count': 46, 'created_at': 'Wed Jul 24 13:31:10 +0000 2013', 'favourites_count': 2421, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 2431, 'lang': None, 'status': {'created_at': 'Fri May 24 22:22:09 +0000 2019', 'id': 1132049171119312896, 'id_str': '1132049171119

In [5]:
c_id = pd.Series(c_id)
f_id = pd.Series(f_id)
f_name = pd.Series(f_name)
f_screen_name = pd.Series(f_screen_name)
f_description = pd.Series(f_description)
f_verified = pd.Series(f_verified)
f_following = pd.Series(f_following)
follower = pd.concat([c_id, f_id, f_name, f_screen_name, f_description, f_verified, f_following], axis=1)
follower.columns =['ceo_id', 'f_id', 'f_name', 'f_screen_name', 'f_description', 'f_verified' ,'f_following']

pd.set_option('display.max_colwidth', -1)
follower.head(10)

,ceo_id,f_id,f_name,f_screen_name,f_description,f_verified
0,244647486,2988364746,NterpriseIT,NterpriseIT,Helping you navigate the maze of B2B IT,False
1,244647486,1102403520693760001,FranciscaSesi,Sesi Francisca Mamattah.,"Fashion Designer.....Single mum....have a beautiful Daughter, Alice.......from Accra...Ghana... West Africa....i love movies.... comedy...music and politics.",False
2,244647486,997005996336836608,rafakhaliffa,Rafi Khaliffa,caballero de la noche,False
3,244647486,725254122132365312,softermii,Softermii,"Softermii provides full cycle of software development services, with main focus on Microsoft stack, Ruby on Rails, Python and Mobile technologies.",False
4,244647486,1067803633104896002,duckworth_nigel,Nigel Duckworth,https://t.co/iUxZuqJrFG | Creative + Critical Thinker | Business + Technology #BusinessStrategy #DigitalTransformation #AI #Blockchain #SupplyChain,False
5,244647486,183343331,geoffholm,Geoff Holm,I can't count the reasons I should stay #SWCT: NDODPGK16.\n\nAll tweets and opinions my own...you know the drill.,False
6,244647486,32409105,austegard,Oskar,,False
7,244647486,68261888,asalcedo__,Alfredo Salcedo,"#Startups | Currently @LootApp | ex @Amazon, ex mgmt consultant | wannabe entrepreneur and part-time nerd | MBA @LBS | coffee addict, gamer, beer junkie, #tech",False
8,244647486,478139803,tamarafjohn,Tamara (Ford) John,"Bookworm, coffee addict, and big fan of trees and leaves. Mostly #data #analytics tweets. Corp Comm @lookerdata. #Sacramentoproud✌️☮ Views are my own.",False
9,244647486,3647563642,MeTienenHartoAR,Me Tienen Harto,"Estudiante de Seguridad, 22, CABA. Por ahora apoyo a Espert y Lopérfido, pero soy todavía más reformista y quiero empezar el país de nuevo.",False


In [7]:

logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
follower.to_gbq('ceo_tweet_streaming.follower_description', 'ra-work-content')
#following_description

40 out of 40 rows loaded.
40 out of 40 rows loaded.


## following

In [5]:
def implicit():
    from google.cloud import storage
    storage_client = storage.Client()
    buckets = list(storage_client.list_buckets())
    print(buckets)

client = bigquery.Client()
dataset_id = 'ceo_tweet_streaming'  
table_id = 'following' 
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref) 

def store_data(c_id, f_id, f_name, f_screen_name, f_description, f_verified, f_following):

        rows_to_insert = [(c_id, f_id, f_name, f_screen_name, f_description, f_verified, f_following), ]
        errors = client.insert_rows(table, rows_to_insert)
        print(errors, 'bq')    
        assert errors == []
        return

In [11]:
cnt = 0
fail = 0
for index, row in ceo.iterrows():
    id = row['id']   
    try: 
        cnt += 1
        tweet = tweepy.Cursor(api.friends, user_id=id,  wait_on_rate_limit =True, wait_on_rate_limit_notify=True).items(20)
        #api.friends
        if tweet == []:
            continue
        if cnt == 20:
            break
        else: 
            for line in tweet:
                store_data(id, line.id, line.screen_name, line.name, line.description, line.verified, line.following)

    except tweepy.TweepError:
        fail += 1
print('search fail :' + str(fail))
print('return search result :' + str(cnt))

[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] b

Rate limit reached. Sleeping for: 776


[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
[] bq
search fail :1
return search result :20
